# Classification d'opinion avec un Transformer 

Téléchargez les données sur le site d'origine:
[https://ai.stanford.edu/~amaas/data/sentiment/]

Penser à dezipper le fichier d'embeddings présent dans le répertoire data.

Ensuite, le TP est opérationnel :)

In [ ]:
# import standard + 
# 
import numpy as np

import torch.nn.functional as F
import torch
import torch.nn as nn
from tqdm.autonotebook import tqdm
from torch.utils.data import Dataset, DataLoader

import os
import time
import logging
import re
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = "cpu"
print(device)



In [ ]:
from pathlib import Path
from IPython.display import display, HTML
from torch.utils.tensorboard import SummaryWriter

# Chemin vers TensorBoard
TB_PATH = "/tmp/logs/module-Transf"

# TENSORBOARD 
# usage externe de tensorboard: (1) lancer la commande dans une console; (2) copier-coller l'URL dans un navigateur
display(HTML("<h2>Informations</h2><div>Pour visualiser les logs, tapez la commande : </div>"))
print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")
print("Une fois la commande lancer dans la console, copier-coller l'URL dans votre navigateur")



A. Chargement des données
------------------

Tout le code est fourni. Le cadre est le même que pour la classification de noms: many-to-one. La tâche est de la classification d'opinion (sentiment en anglais)



In [ ]:
GLOVE_PATH = Path("data/glove")
DATASET_PATH_TRAIN = Path("data/aclImdb/train")
DATASET_PATH_TEST = Path("data/aclImdb/test")

NB_DOC_MAX = 12500 # par classe
IMDB_CLASSES  = ['neg','pos']
VOC_SIZE = 10000
BATCH_SIZE = 32
MAX_CHAR_SIZE = 1000


labels = dict(zip(IMDB_CLASSES,[0,1]))

def load_data(datapath, classes, max_size=None):
    txts = []
    files = []
    filelabels = []
    for label in classes:
        c = 0
        new = [os.path.join(datapath / label, f) for f in os.listdir(datapath / label) if f.endswith(".txt")]
        files += new
        # filelabels += [labels[label]] * len(new) 
        for file in (datapath / label).glob("*.txt"):
            t = file.read_text()
            txts.append(t if len(t)<MAX_CHAR_SIZE else t[:MAX_CHAR_SIZE])
            filelabels.append(labels[label])
            c+=1
            if max_size !=None and c>=max_size: break

    return txts, files, filelabels
    #     c+=1
    #     if train_max_size !=None and c>train_max_size: break


txts, files, filelabels = load_data(DATASET_PATH_TRAIN, IMDB_CLASSES, max_size = NB_DOC_MAX)
txts_test, files_test, filelabels_test = load_data(DATASET_PATH_TEST, IMDB_CLASSES, max_size = NB_DOC_MAX)

In [ ]:
print(files[0])
print(txts[0])
print(filelabels[0])
print(len(files),len(txts),len(filelabels))
# labels

## Construction du vocabulaire

On introduit un *tokenizer* pour découper le texte en groupe de lettre. Ce découpage est non trivial (cf cours), on utilise un composant d'analyse venant des libs huggingface

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialiser un tokenizer WordPiece
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

# Définir les pré-traitements
tokenizer.pre_tokenizer = Whitespace()

# Créer un entraîneur avec un vocabulaire cible
trainer = WordPieceTrainer(
    vocab_size=VOC_SIZE,  # Limite du vocabulaire
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

In [ ]:
tokenizer.train(files, trainer)

In [ ]:
# Exemple de tokenisation
output = tokenizer.encode("This is an example.", add_special_tokens=True)
print("Tokens:", output.tokens)
print("Token IDs:", output.ids)

# Autre exemple de tokenisation avec des mots plus rares
output = tokenizer.encode("Exemple en francais, loin de la base IMDB")
print("Tokens:", output.tokens)
print("Token IDs:", output.ids)

Récupération des indices de tokens spéciaux

In [ ]:
PAD = tokenizer.encode("[PAD]").ids[0]
print("PAD",PAD)

CLS = tokenizer.encode("[CLS]").ids[0]
print("CLS",CLS)

EOS = tokenizer.encode(".").ids[0]
print("EOS", EOS)

In [ ]:
# fabrication de tous les codes (+ astuce pour ajouter les CLS)

allcodes = [torch.tensor(tokenizer.encode("[CLS] " + p).ids) for p in txts]
allcodes_test = [torch.tensor(tokenizer.encode("[CLS] " + p).ids) for p in txts_test]

### Chaine de traitement

1. Récupération des données avec un Dataset
2. Construction du dataloader

On se place dans une logique générative: on ne fait pas d'ensemble de test (c'est discutable)


In [ ]:
from torch.utils.data import Dataset, DataLoader
import sys
import re

class TextDataset(Dataset):
    def __init__(self, texts: list, labels):
        self.labels = labels
        self.phrasesnum = texts

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.phrasesnum[i], torch.tensor(self.labels[i])



In [ ]:
ds_train = TextDataset(allcodes,filelabels)
ds_test  = TextDataset(allcodes_test,filelabels_test)

In [ ]:

print("longueur:", len(ds_train))
print("doc 0: ", ds_train[0])
print("doc 1: ", ds_train[1])
print("doc 2: ", ds_train[2])

# savez-vous retrouver les token EOS/.?

In [ ]:
from torch.nn.utils.rnn import pad_sequence


def collate_fn(batch):
    sequences, labels = zip(*batch)
    lengths = [len(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=False)
    return padded_sequences, torch.tensor(lengths), torch.tensor(labels)

train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)



In [ ]:
# Test

batch =next(iter( train_loader))
padded_sequences, lengths, labels = batch
print("Padded sequences:", padded_sequences)
print(padded_sequences.size())
# print("Lengths:", lengths)
print("Labels:", labels)


## C. Création du réseau

1. Comprendre les positional embeddings
    - prendre le temps d'afficher les dimensions et de comprendre comment ils vont être utilisés
    - activer ou ne pas activer le gradient???
    
2. Construire le réseau

In [ ]:
import math
def generate_sinusoidal_embeddings(seq_len, d_model):
    position = torch.arange(seq_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(seq_len, d_model)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

In [ ]:
import matplotlib.pyplot as plt

pe = generate_sinusoidal_embeddings(100, 128)

plt.figure()
plt.imshow(pe.numpy())


In [ ]:

class TransSent(nn.Module):
    def __init__(self, emb_size, voc_size, num_layers, num_heads, hidden_size_mlp , output_size, maxlen=1000):
        super(TransSent, self).__init__()

        self.emb_size = emb_size

        self.emb = nn.Embedding(voc_size, emb_size)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_size, 
            nhead=num_heads, 
            dim_feedforward=hidden_size_mlp,
            activation='relu'
        )

        # Création d'un TransformerEncoder avec plusieurs couches
        self.trans = nn.TransformerEncoder(
            self.encoder_layer, 
            num_layers=num_layers
        )

        # Attention, seuls les modules sont envoyés vers les devices
        # pour envoyer automatiquement les tenseurs, il faut les "enregistrer"
        self.register_buffer("posemb", generate_sinusoidal_embeddings(maxlen, self.emb_size).unsqueeze(1))

        # du CLS vers la classif
        self.h2o = nn.Linear(emb_size, output_size)

   
    def forward(self, input, lengths=None):
        # Principales étapes
        # 1. translation of the input from int to emb
        # 2. Passage dans le trans
        # 3. Prediction sur le CLS

        # print("input", input.size())
        maxlen = input.size(0)
        batch_size = input.size(1)

        # A analyser (et à utiliser plus tard)
        padding_mask = (input[:, :] == PAD).T 

        # 1. translation of the input from int to emb + ajout des positional embeddings
        #  TODO 
        
        # 2. Passage dans le transformer... Avec le masque pour le padding
        encoded_output = self.trans(xemb, src_key_padding_mask=padding_mask)
        # print("encoded_output", encoded_output.size())
        
        # 3. Appliquer la classification sur le CLS
        #  TODO 
        
        return output



In [ ]:

# choose hidden size
emb_size = 128
voc_size = VOC_SIZE
num_layers = 4
num_heads = 4
hidden_size_mlp = 128
output_size = 2
# build network
transf = TransSent( emb_size, voc_size, num_layers, num_heads, hidden_size_mlp , output_size)
transf.name = "TransSent-"+time.asctime()

In [ ]:
loss = nn.CrossEntropyLoss()
# batch
x, lengths, y = next(iter(train_loader))
print(x.size(),y.size())


yhat = transf(x)
print(yhat.size())
l = loss(yhat,y)

## C. Training

1. put the data into a DataLoader
2. choose a loss function 
3. run a standard training loop

In [ ]:
# définition de la métrique d'évaluation
def accuracy(yhat,y):
    # y encode les indexes, s'assurer de la bonne taille de tenseur
    assert len(y.shape)==1 or y.size(1)==1
    return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).float().mean()

In [ ]:

    
def train(model,epochs,train_loader,test_loader):
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    optim = torch.optim.Adam(model.parameters(),lr=5e-4)    # choix optimizer
    model = model.to(device)
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()                            # choix loss
    # 
    # loss = nn.CrossEntropyLoss(weight=cl_weight.to(device))                            # choix loss
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x, lengths, y in train_loader:                            # boucle sur les batchs
            optim.zero_grad()
            x,y = x.to(device), y.to(device)                # y doit être un tensor (pas un int)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)                             # attention, il peut y avoir un batch + petit (le dernier)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)
        if epoch % 2 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x, lengths, y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)


In [ ]:
# ~10 minutes sur CPU
n_epoch = 20
train(transf, n_epoch, train_loader, test_loader)


In [ ]:
import os

def save_model(model,fichier): # pas de sauvegarde de l'optimiseur ici
      """ sauvegarde du modèle dans fichier """
      state = {'model_state': model.state_dict()}
      torch.save(state,fichier) # pas besoin de passer par pickle
 
def load_model(fichier,model):
      """ Si le fichier existe, on charge le modèle  """
      if os.path.isfile(fichier):
          state = torch.load(fichier)
          model.load_state_dict(state['model_state'])
      else:
           print("Erreur de chargement du fichier")

In [ ]:
# sauvegarde du modèle

path = "./"
fichier = path+f"model/{transf.name}"
save_model(transf.to("cpu"),fichier)

In [ ]:

# Creation de la coquille pour charger le modèle
emb_size = 128
voc_size = VOC_SIZE
num_layers = 1
num_heads = 1
hidden_size_mlp = 128
output_size = 2
# build network
transf = TransSent( emb_size, voc_size, num_layers, num_heads, hidden_size_mlp , output_size)

# chargement
path = "./"
fichier = path+f"model/TransSent-1H-1L"

load_model(fichier,transf)

## D. Amélioration de l'architecture et analyse des résultats

1. Tester l'augmentation du nombre de têtes
2. Analyse la forme des attentions, comment trouver les mots qui contrinuent le plus au diagnostic?
3. Utiliser TensorBoard pour visualiser les représentations de mots

In [ ]:
# essais qualitatifs
# tester le modèle sur des exemples de chaines de caractères 
# (oblige à faire la tokenisation + CLS)



In [ ]:
# Afficher le texte des messages sur lesquels on fait des erreurs

In [ ]:
# Afficher la matrice d'attention (ou seulement la ligne CLS) pour une entrée

In [ ]:
# utiliser tensorboard pour afficher en 2D les embeddings des mots

# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###